En faisant le produit avec la matrice adaptative si jamais un pays se retrouve seul dans une classe, toute la colonne de cette classe se retrouve à zéro.

In [113]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from adjency_matrix import AdaptativeMatrix, DTW,AgeReduction
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.decomposition import PCA

In [114]:
AM = AdaptativeMatrix()
dtw = DTW()
AR = AgeReduction()

In [78]:
def adjency_matrix(X: torch.Tensor, coord_countries: torch.Tensor):
    """
    X is 3D tensor
    coord_countries is 2D matrix contains longitude and latitude of each country. 1st column represents long and 2nd represents lat
    """
    n_countries  = X.shape[0]
    clusters = AM.adaptative_matrix(X)
    dtw_matrix = dtw.DTW_Matrix(X)
    long_lat_matrix = np.empty((n_countries, n_countries))
    for i in range(n_countries):
        for j in range(n_countries):
            long_lat_matrix[i,j] = torch.sqrt(torch.mean((coord_countries[i]-coord_countries[j])**2))
    
    # 
    adjency_mat = dtw_matrix * long_lat_matrix
    #print("adj_brut_matrix \n",adjency_mat)
    for i in range(n_countries):
        for j in range(n_countries):
            if clusters[i] != clusters[j]:
                adjency_mat[i,j] = 0.0
    
    #print("long_lat \n",long_lat_matrix)
    print("clusters \n",clusters)
    return adjency_mat


In [12]:
coord_countries = torch.rand(8,2)
X = torch.rand((8,5,3))


In [127]:
AM.adaptative_matrix(X)

The variance explained by principal components:  [0.60030442]
The variance explained by principal components:  [0.75690019]
The variance explained by principal components:  [0.71084506]
The variance explained by principal components:  [0.68386386]
The variance explained by principal components:  [0.78910496]
The variance explained by principal components:  [0.55995661]
The variance explained by principal components:  [0.64140283]
The variance explained by principal components:  [0.84214969]


array([1, 2, 0, 1, 2, 0, 1, 2], dtype=int32)

In [133]:
K = AR.age_reduction(X)

The variance explained by principal components:  [0.60030442]
The variance explained by principal components:  [0.75690019]
The variance explained by principal components:  [0.71084506]
The variance explained by principal components:  [0.68386386]
The variance explained by principal components:  [0.78910496]
The variance explained by principal components:  [0.55995661]
The variance explained by principal components:  [0.64140283]
The variance explained by principal components:  [0.84214969]


In [138]:
km = KMeans(n_clusters=3,random_state=0)
km.fit(K)

KMeans(n_clusters=3, random_state=0)

In [139]:
km.get_params()

{'algorithm': 'lloyd',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 300,
 'n_clusters': 3,
 'n_init': 'auto',
 'random_state': 0,
 'tol': 0.0001,
 'verbose': 0}

In [98]:
coord_countries

tensor([[0.2868, 0.5596],
        [0.5044, 0.9553],
        [0.5897, 0.2719],
        [0.8361, 0.7651],
        [0.6289, 0.3322],
        [0.1806, 0.3381],
        [0.5337, 0.3976],
        [0.9547, 0.3576]])

In [99]:
X

tensor([[[0.9538, 0.4499, 0.3036],
         [0.8550, 0.6066, 0.6212],
         [0.7290, 0.2463, 0.7476],
         [0.7651, 0.5005, 0.1388],
         [0.2775, 0.2394, 0.5119]],

        [[0.1048, 0.1773, 0.7183],
         [0.4228, 0.9801, 0.0770],
         [0.3712, 0.5383, 0.7897],
         [0.4925, 0.5857, 0.0366],
         [0.8549, 0.9178, 0.3294]],

        [[0.0303, 0.6619, 0.2996],
         [0.5467, 0.4159, 0.4198],
         [0.9513, 0.2092, 0.8401],
         [0.3657, 0.3834, 0.4449],
         [0.7616, 0.6635, 0.0736]],

        [[0.7350, 0.2804, 0.6159],
         [0.7631, 0.7002, 0.5183],
         [0.3557, 0.8965, 0.0207],
         [0.5160, 0.6266, 0.6134],
         [0.9134, 0.8924, 0.2442]],

        [[0.0732, 0.0088, 0.9674],
         [0.1410, 0.5433, 0.1109],
         [0.4854, 0.2945, 0.3443],
         [0.1957, 0.5116, 0.4359],
         [0.2095, 0.1666, 0.3481]],

        [[0.5879, 0.9253, 0.4465],
         [0.8000, 0.3759, 0.3387],
         [0.2139, 0.7806, 0.0402],
         [

In [102]:
adjency_matrix(X,coord_countries)

The variance explained by principal components:  [0.60030442]
The variance explained by principal components:  [0.75690019]
The variance explained by principal components:  [0.71084506]
The variance explained by principal components:  [0.68386386]
The variance explained by principal components:  [0.78910496]
The variance explained by principal components:  [0.55995661]
The variance explained by principal components:  [0.64140283]
The variance explained by principal components:  [0.84214969]
The variance explained by principal components:  [0.60030442]
The variance explained by principal components:  [0.75690019]
The variance explained by principal components:  [0.71084506]
The variance explained by principal components:  [0.68386386]
The variance explained by principal components:  [0.78910496]
The variance explained by principal components:  [0.55995661]
The variance explained by principal components:  [0.64140283]
The variance explained by principal components:  [0.84214969]
clusters

tensor([[0.0000, 0.1630, 0.0000, 0.0717, 0.1812, 0.0000, 0.0570, 0.0773],
        [0.1630, 0.0000, 0.0000, 0.1041, 0.1737, 0.0000, 0.0753, 0.1167],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0717, 0.1041, 0.0000, 0.0000, 0.1327, 0.0000, 0.0781, 0.0889],
        [0.1812, 0.1737, 0.0000, 0.1327, 0.0000, 0.0000, 0.0356, 0.1597],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0570, 0.0753, 0.0000, 0.0781, 0.0356, 0.0000, 0.0000, 0.0538],
        [0.0773, 0.1167, 0.0000, 0.0889, 0.1597, 0.0000, 0.0538, 0.0000]],
       dtype=torch.float64)